In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from torch import nn

df = pd.read_csv("output_adam.csv")
df_label = pd.read_csv("mirai_labels.csv")
df["alert"] = df_label
df.head()

,frame.number,frame.len,frame.time,frame.time_epoch,frame.protocols,eth.src,eth.dst,eth.type,ip.src,ip.dst,...,imap.request.command,imap.response,ftp.request.command,ftp.request.arg,ftp.response.code,ftp.response.arg,ipv6.src,ipv6.dst,ipv6.plen,alert
0,1,60,"Oct 25, 2018 07:46:22.933899000 CEST",1.540446e+09,eth:ethertype:ip:tcp,5c:cf:7f:05:f2:c6,4c:09:d4:c6:12:7b,0x0800,192.168.2.108,52.24.43.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2,60,"Oct 25, 2018 07:46:22.933904000 CEST",1.540446e+09,eth:ethertype:ip:tcp,5c:cf:7f:05:f2:c6,4c:09:d4:c6:12:7b,0x0800,192.168.2.108,52.25.66.250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,3,86,"Oct 25, 2018 07:46:22.934426000 CEST",1.540446e+09,eth:ethertype:ip:icmp:ip:tcp,4c:09:d4:c6:12:7b,5c:cf:7f:05:f2:c6,0x0800,192.168.2.1,192.168.2.108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,4,86,"Oct 25, 2018 07:46:22.934636000 CEST",1.540446e+09,eth:ethertype:ip:icmp:ip:tcp,4c:09:d4:c6:12:7b,5c:cf:7f:05:f2:c6,0x0800,192.168.2.1,192.168.2.108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,5,60,"Oct 25, 2018 07:46:23.291054000 CEST",1.540446e+09,eth:ethertype:arp,48:02:2e:01:83:15,ff:ff:ff:ff:ff:ff,0x0806,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764137 entries, 0 to 764136
Data columns (total 74 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   frame.number                764137 non-null  int64  
 1   frame.len                   764137 non-null  int64  
 2   frame.time                  764137 non-null  object 
 3   frame.time_epoch            764137 non-null  float64
 4   frame.protocols             764137 non-null  object 
 5   eth.src                     764137 non-null  object 
 6   eth.dst                     764137 non-null  object 
 7   eth.type                    760438 non-null  object 
 8   ip.src                      197701 non-null  object 
 9   ip.dst                      197701 non-null  object 
 10  ip.len                      197701 non-null  float64
 11  ip.ttl                      197701 non-null  float64
 12  ip.flags                    197701 non-null  object 
 13  ip.frag_offset

In [4]:
df["ip.dst"].unique()

array(['52.24.43.67', '52.25.66.250', '192.168.2.108', nan,
       '192.168.2.110', '192.168.2.101', '192.168.2.1', '192.168.2.115',
       '114.114.114.114', '50.19.254.134', '122.248.234.207',
       '46.137.188.54', '122.226.84.253', '61.188.37.216',
       '120.24.59.150', '114.215.137.159', '104.199.156.58',
       '192.168.2.113', '239.255.255.250', '192.168.2.112',
       '74.125.31.99', '192.168.2.109', '8.8.8.8', '112.74.102.136',
       '54.154.5.26', '54.149.224.179', '192.168.2.126', '210.61.248.232',
       '42.99.254.162', '203.69.81.91', '61.220.62.219', '119.29.29.29',
       '255.255.255.255', '224.0.0.22', '192.168.1.252', '112.74.124.169',
       '192.168.2.119', '133.100.9.2', '192.168.2.196', '139.78.100.163',
       '131.107.1.10', '199.165.76.11', '140.142.16.34', '128.138.140.44',
       '137.146.210.250', '192.36.144.22', '192.168.2.117', '129.7.1.66',
       '192.43.244.18', '158.121.104.4', '192.168.2.120', '192.6.38.127',
       '216.133.140.77', '140.221.8.

In [2]:


def ordinalize_packets(packet_data, column="ip"):
    src_col = f"{column}.src"
    dst_col = f"{column}.dst"
    if 'frame.number' not in packet_data.columns or src_col not in packet_data.columns or dst_col not in packet_data.columns:
        raise ValueError("DataFrame must contain 'frame.number' and  columns")

    packet_data = packet_data.sort_values('frame.number').copy()

    ip_to_ordinal = {}
    next_ordinal = 1
    results = []

    for index, row in packet_data.iterrows():
        frame_num = row['frame.number']
        src = row[src_col]
        dst = row[dst_col]
        # Skip rows without IP source (like ARP packets)
        if pd.isna(src):
            continue

        # Assign ordinal for this packet
        if src not in ip_to_ordinal:
            ip_to_ordinal[src] = next_ordinal
            next_ordinal += 1
        if dst not in ip_to_ordinal:
            ip_to_ordinal[dst] = next_ordinal
            next_ordinal += 1

        results.append({
            'frame.number': frame_num,
            'ordinal_src': ip_to_ordinal[src],
            'ordinal_dst': ip_to_ordinal[dst],
        })

    result_df = pd.DataFrame(results)

    final_df = pd.merge(packet_data, result_df, on=['frame.number'], how='left')
    final_df[[src_col, dst_col]] = final_df[["ordinal_src", "ordinal_dst"]]
    return final_df.drop(columns=["ordinal_src", "ordinal_dst"])


result_df = ordinalize_packets(df, column="ip")

# Save to a new CSV file


#print(f"Processed {len(result_df)} packets")
print(result_df[['frame.number', 'ip.src', 'ip.dst']].head(20))

    frame.number  ip.src  ip.dst
0              1     1.0     2.0
1              2     1.0     3.0
2              3     4.0     1.0
3              4     4.0     1.0
4              5     NaN     NaN
5              6     5.0     6.0
6              7     6.0     5.0
7              8     7.0     4.0
8              9     4.0     7.0
9             10     1.0     2.0
10            11     1.0     3.0
11            12     4.0     1.0
12            13     7.0     4.0
13            14     8.0     4.0
14            15     6.0     9.0
15            16     4.0     6.0
16            17    10.0    11.0
17            18    10.0    12.0
18            19    10.0    13.0
19            20    10.0    14.0


In [3]:
features = ["frame.number", "frame.len", 'frame.time', 'eth.src', 'eth.dst', 'ip.src', 'ip.dst', 'ip.len', 'ip.ttl',
            'tcp.srcport', 'tcp.dstport',
            'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.reset', "alert", "udp.srcport", "udp.dstport"]
df = df[features]
df["frame.len"] /= 65535
df["ip.len"] /= 65535
df["ip.ttl"] /= 255
df["tcp.srcport"] /= 65535
df["tcp.dstport"] /= 65535
df["udp.srcport"] /= 65535
df["udp.dstport"] /= 65535
df["frame.time"] = df["frame.time"].str.replace(" CEST", "")
df["frame.time"] = pd.to_datetime(df["frame.time"])
df['dow_sin'] = np.sin(2 * np.pi * df["frame.time"].dt.dayofweek / 6)
df['dow_cos'] = np.cos(2 * np.pi * df["frame.time"].dt.dayofweek / 6)
df['hour_sin'] = np.sin(2 * np.pi * df["frame.time"].dt.hour / 24.0)
df['hour_cos'] = np.cos(2 * np.pi * df["frame.time"].dt.hour / 24.0)
df["minute_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.minute / 60)
df["minute_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.minute / 60)
df["second_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.second / 60)
df["second_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.second / 60)
df["ms_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.microsecond / 1000000)
df["ms_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.microsecond / 1000000)
df = df.fillna(0)  # only tcp is left
df = df.drop(columns=["frame.time"])

In [15]:
df.head()

,frame.number,frame.len,eth.src,eth.dst,ip.src,ip.dst,ip.len,ip.ttl,tcp.srcport,tcp.dstport,...,dow_sin,dow_cos,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos,ms_sin,ms_cos
0,1,0.000916,5c:cf:7f:05:f2:c6,4c:09:d4:c6:12:7b,192.168.2.108,52.24.43.67,0.000671,1.00000,0.321569,0.001221,...,1.224647e-16,-1.0,0.965926,-0.258819,-0.994522,0.104528,0.743145,-0.669131,-0.403487,0.914985
1,2,0.000916,5c:cf:7f:05:f2:c6,4c:09:d4:c6:12:7b,192.168.2.108,52.25.66.250,0.000671,1.00000,0.313298,0.126345,...,1.224647e-16,-1.0,0.965926,-0.258819,-0.994522,0.104528,0.743145,-0.669131,-0.403458,0.914998
2,3,0.001312,4c:09:d4:c6:12:7b,5c:cf:7f:05:f2:c6,192.168.2.1,192.168.2.108,0.001099,0.25098,0.321569,0.001221,...,1.224647e-16,-1.0,0.965926,-0.258819,-0.994522,0.104528,0.743145,-0.669131,-0.400455,0.916316
3,4,0.001312,4c:09:d4:c6:12:7b,5c:cf:7f:05:f2:c6,192.168.2.1,192.168.2.108,0.001099,0.25098,0.313298,0.126345,...,1.224647e-16,-1.0,0.965926,-0.258819,-0.994522,0.104528,0.743145,-0.669131,-0.399246,0.916844
4,5,0.000916,48:02:2e:01:83:15,ff:ff:ff:ff:ff:ff,0,0,0.000000,0.00000,0.000000,0.000000,...,1.224647e-16,-1.0,0.965926,-0.258819,-0.994522,0.104528,0.669131,-0.743145,0.966915,-0.255099


In [4]:
from torch.utils.data import Dataset


class NetworkDataset(Dataset):
    def __init__(self, data, window=128):
        self.data = data
        self.window = window

    def __getitem__(self, index):
        packets = self.data[index:index + self.window]
        labels = packets['alert'].to_numpy().astype(np.float32)
        packets = ordinalize_packets(packets, column="ip")
        packets = ordinalize_packets(packets, column="eth")
        packets = packets.drop(columns=['alert', "frame.number"]).fillna(0)
        return (packets[["eth.src", "eth.dst"]].astype(int).values, packets[["ip.src", "ip.dst"]].astype(int).values,
                packets.drop(columns=["ip.src", "ip.dst", "eth.src", "eth.dst"]).values.astype("float32"), labels)

    def __len__(self):
        return self.data.shape[0] - self.window

In [100]:
nds = NetworkDataset(df, 1024)
nds[121000][0].shape

(1024, 2)

In [72]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.eth_embed = nn.Embedding(48, 32)
        self.ip_embed = nn.Embedding(48, 32)
        self.total_embed = nn.Linear(148, 64)
        self.conv = nn.Conv2d(1, 1, kernel_size=3, padding=1)
        self.projection = nn.Linear(64, 1)
        
    def forward(self, eth, ip, packet):
        B = packet.size(0)
        L = packet.size(1)
        eth = self.eth_embed(eth).view(B, L, -1)
        ip = self.ip_embed(ip).view(B, L, -1)
        pk_embed = self.total_embed(torch.cat((eth, ip, packet), dim=-1))
        processed = nn.functional.relu(self.conv(pk_embed.unsqueeze(1))).squeeze(1)
        preds = self.projection(processed).squeeze(-1)
        return nn.functional.sigmoid(preds)

In [73]:
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
from accelerate import Accelerator

accelerator = Accelerator(mixed_precision="bf16")
model = Model()
dataset = NetworkDataset(df[:200000], 128)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True, num_workers=24, pin_memory=True, persistent_workers=True)
optimizer = Adam(model.parameters(), lr=1e-4)
loss_fn = nn.BCELoss()

model, dataloader, optimizer = accelerator.prepare(model, dataloader, optimizer)


for epoch in range(20):
    count = 0
    total = 0
    
    for eth, ip, pkt, labels in tqdm(dataloader):
        optimizer.zero_grad()
        pkt = model(eth, ip, pkt)
        loss = loss_fn(pkt, labels)
        count += 1
        total += loss.item()
        loss.backward()
        optimizer.step()
        
    print("Epoch {} Loss {}".format(epoch, total/count))

100%|██████████| 3123/3123 [01:11<00:00, 43.49it/s]


Epoch 0 Loss 0.18315705482609715


  9%|▉         | 291/3123 [00:07<01:10, 40.25it/s]
Exception in thread Thread-27 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 59, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 35, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
         ^^^^^^^^^^^
  F

KeyboardInterrupt: 

In [74]:
ip, eth, pkt, labels = dataset[0]
ip = torch.from_numpy(ip).cuda().unsqueeze(0)
eth = torch.from_numpy(eth).cuda().unsqueeze(0)
pkt = torch.from_numpy(pkt).cuda().unsqueeze(0)
labels = torch.from_numpy(labels).cuda().unsqueeze(0)

pred = model(ip, eth, pkt)
pred

tensor([[3.7079e-03, 4.4441e-04, 7.3242e-04, 3.8300e-03, 9.9945e-04, 4.6082e-03,
         6.0654e-04, 1.3657e-03, 2.0313e-04, 1.6880e-04, 3.6812e-04, 3.3569e-04,
         1.7929e-04, 1.0872e-04, 1.7929e-04, 1.1587e-04, 1.2302e-04, 1.0872e-04,
         1.4877e-04, 7.9632e-05, 9.6321e-05, 6.6280e-05, 7.0095e-05, 1.7929e-04,
         5.8746e-04, 3.8300e-03, 3.8300e-03, 3.8300e-03, 3.8300e-03, 2.1648e-04,
         9.0122e-05, 1.0872e-04, 1.4019e-04, 9.0122e-05, 1.5831e-04, 1.0872e-04,
         2.3079e-04, 1.9073e-04, 1.4019e-04, 1.9073e-04, 1.0872e-04, 1.3161e-04,
         8.4877e-05, 9.0122e-05, 6.8665e-04, 2.1210e-03, 8.8120e-04, 2.7847e-04,
         1.2302e-04, 1.4877e-04, 9.6321e-05, 5.8746e-04, 8.2779e-04, 3.2806e-03,
         3.6011e-03, 5.8746e-04, 5.5313e-04, 1.4019e-04, 1.0252e-04, 2.4605e-04,
         2.0313e-04, 1.2302e-04, 9.6321e-05, 6.6280e-05, 7.0095e-05, 6.1989e-05,
         8.4877e-05, 1.0252e-04, 1.2302e-04, 1.2302e-04, 1.0872e-04, 2.1648e-04,
         2.1648e-04, 2.1648e

In [68]:
loss_fn(pred.reshape(-1, 1), labels.reshape(-1, 1))

tensor(0.7049, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [75]:
pred

tensor([[3.7079e-03, 4.4441e-04, 7.3242e-04, 3.8300e-03, 9.9945e-04, 4.6082e-03,
         6.0654e-04, 1.3657e-03, 2.0313e-04, 1.6880e-04, 3.6812e-04, 3.3569e-04,
         1.7929e-04, 1.0872e-04, 1.7929e-04, 1.1587e-04, 1.2302e-04, 1.0872e-04,
         1.4877e-04, 7.9632e-05, 9.6321e-05, 6.6280e-05, 7.0095e-05, 1.7929e-04,
         5.8746e-04, 3.8300e-03, 3.8300e-03, 3.8300e-03, 3.8300e-03, 2.1648e-04,
         9.0122e-05, 1.0872e-04, 1.4019e-04, 9.0122e-05, 1.5831e-04, 1.0872e-04,
         2.3079e-04, 1.9073e-04, 1.4019e-04, 1.9073e-04, 1.0872e-04, 1.3161e-04,
         8.4877e-05, 9.0122e-05, 6.8665e-04, 2.1210e-03, 8.8120e-04, 2.7847e-04,
         1.2302e-04, 1.4877e-04, 9.6321e-05, 5.8746e-04, 8.2779e-04, 3.2806e-03,
         3.6011e-03, 5.8746e-04, 5.5313e-04, 1.4019e-04, 1.0252e-04, 2.4605e-04,
         2.0313e-04, 1.2302e-04, 9.6321e-05, 6.6280e-05, 7.0095e-05, 6.1989e-05,
         8.4877e-05, 1.0252e-04, 1.2302e-04, 1.2302e-04, 1.0872e-04, 2.1648e-04,
         2.1648e-04, 2.1648e